<a href="https://colab.research.google.com/github/PedroJoas/SQL-analyses/blob/main/coffee%20sales/Coffe_Sales_Analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On kaggle i make a notebook with same analyses using python

[Click here](https://www.kaggle.com/code/pedrojos/coffee-sales-analyses)

In [6]:
!pip install opendatasets --upgrade --quiet

In [13]:
import pandas as pd
import opendatasets as od
import sqlite3 as sql

# Connection db

In [3]:
conn = sql.connect('coffeeSales.db')
print("Opened database successfully");

Opened database successfully


In [4]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS coffeSale (
    date DATE NOT NULL,
    datetime DATE NOT NULL,
    cash_type TEXT NOT NULL,
    card TEXT,
    money REAL NOT NULL,
    coffee_name TEXT NOT NULL
)
''')

conn.commit()

print("Table created successfully")

Table created successfully


# Download data

You need a kaggle account to use this code

This [medium article](https://ravi-chan.medium.com/how-to-download-any-data-set-from-kaggle-7e2adc152d7f#c629) explains about that







In [8]:
dataset = 'https://www.kaggle.com/datasets/ihelon/coffee-sales'
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: pedrojos
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ihelon/coffee-sales


100%|██████████| 10.4k/10.4k [00:00<00:00, 2.46MB/s]

In [9]:
coffee = pd.read_csv('coffee-sales/index.csv')
coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         896 non-null    object 
 1   datetime     896 non-null    object 
 2   cash_type    896 non-null    object 
 3   card         807 non-null    object 
 4   money        896 non-null    float64
 5   coffee_name  896 non-null    object 
dtypes: float64(1), object(5)
memory usage: 42.1+ KB


In [10]:
coffee['date'] = pd.to_datetime(coffee['date'])
coffee['datetime'] = pd.to_datetime(coffee['datetime'])
coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         896 non-null    datetime64[ns]
 1   datetime     896 non-null    datetime64[ns]
 2   cash_type    896 non-null    object        
 3   card         807 non-null    object        
 4   money        896 non-null    float64       
 5   coffee_name  896 non-null    object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 42.1+ KB


In [11]:
coffee.to_sql('coffeeSales', conn, if_exists='replace', index=False)

896

# Main questions
1. How much did each card spend?
2. Which was more commonly used, card or cash? And which had the highest
expenditure?
3. Which coffee sold the most? And which generated the most profit?
4. Which day of the week had the highest number of sales? And which had the highest revenue?
5. And which coffee for each day?
6. Which time had the highest number of sales?

In [19]:
query = '''
SELECT card, SUM(money) AS total_spent
FROM coffeeSales
GROUP BY card
order by total_spent DESC
'''
df = pd.read_sql_query(query, conn)
df

,card,total_spent
0,None,3186.00
1,ANON-0000-0000-0012,2297.34
2,ANON-0000-0000-0009,1912.42
3,ANON-0000-0000-0097,681.88
4,ANON-0000-0000-0001,646.14
...,...,...
333,ANON-0000-0000-0316,23.02
334,ANON-0000-0000-0269,23.02
335,ANON-0000-0000-0243,23.02
336,ANON-0000-0000-0202,23.02


In [20]:
query = '''

SELECT cash_type, COUNT(*) AS total_sales
FROM coffeeSales
GROUP BY cash_type
'''
df = pd.read_sql_query(query, conn)
df



,cash_type,total_sales
0,card,807
1,cash,89


In [21]:
query = '''
SELECT cash_type, SUM(money) AS total_sales
FROM coffeeSales
GROUP BY cash_type
'''

df = pd.read_sql_query(query, conn)
df

,cash_type,total_sales
0,card,27406.94
1,cash,3186.00


In [22]:
query = '''
SELECT coffee_name, COUNT(*) AS total_sales
FROM coffeeSales
GROUP BY coffee_name
'''

df = pd.read_sql_query(query, conn)
df

,coffee_name,total_sales
0,Americano,133
1,Americano with Milk,203
2,Cappuccino,164
3,Cocoa,26
4,Cortado,85
5,Espresso,35
6,Hot Chocolate,63
7,Latte,187


In [23]:
query = '''
SELECT coffee_name, SUM(money) AS total_sales
FROM coffeeSales
GROUP BY coffee_name
'''

df = pd.read_sql_query(query, conn)
df

,coffee_name,total_sales
0,Americano,3786.42
1,Americano with Milk,6738.14
2,Cappuccino,6253.50
3,Cocoa,995.66
4,Cortado,2422.80
5,Espresso,827.34
6,Hot Chocolate,2417.46
7,Latte,7151.62
